# Downloading data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications.efficientnet import preprocess_input

# ==== Paths ====
train_dir = '/content/drive/MyDrive/VGG/train'
val_dir   = '/content/drive/MyDrive/VGG/valid'
test_dir  = '/content/drive/MyDrive/VGG/test'

# ==== Parameters ====
img_size    = (256, 256)
batch_size  = 16
seed        = 123
num_classes = 1   # binary classification


In [3]:
def prepare_ds(path):
    ds = tf.keras.utils.image_dataset_from_directory(
        path, seed=seed,
        image_size=img_size, batch_size=batch_size,
        label_mode='int'
    )
    ds = ds.map(lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y))
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = prepare_ds(train_dir)
val_ds   = prepare_ds(val_dir)
test_ds  = prepare_ds(test_dir)


Found 3200 files belonging to 2 classes.
Found 398 files belonging to 2 classes.
Found 402 files belonging to 2 classes.


In [4]:
def build_model(backbone_class):
    backbone = backbone_class(
        input_shape=img_size + (3,),
        include_top=False,
        weights='imagenet'
    )
    backbone.trainable = False

    inputs = layers.Input(shape=img_size + (3,))
    x = backbone(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dropout(0.3, name='dropout1')(x)
    x = layers.Dense(
        128, activation='relu',
        kernel_regularizer=regularizers.l2(1e-4),
        name='dense128'
    )(x)
    x = layers.Dropout(0.3, name='dropout2')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=2.5e-4, weight_decay=1e-4),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
    )
    return model


In [5]:
from tensorflow.keras.applications import EfficientNetB0
model = build_model(EfficientNetB0)
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 8, 8, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling2D)    │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense128 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [6]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping,
    ReduceLROnPlateau, CSVLogger
)

callbacks = [
    ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger('training_log.csv', append=False)
]

history = model.fit(
    train_ds, validation_data=val_ds,
    epochs=50, callbacks=callbacks
)


Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6127 - auc: 0.6697 - loss: 0.6649
Epoch 1: val_auc improved from inf to 0.94638, saving model to best_model.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 652s 3s/step - accuracy: 0.6133 - auc: 0.6704 - loss: 0.6644 - val_accuracy: 0.8492 - val_auc: 0.9464 - val_loss: 0.3846 - learning_rate: 2.5000e-04
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8399 - auc: 0.9079 - loss: 0.4174
Epoch 2: val_auc did not improve from 0.94638
200/200 ━━━━━━━━━━━━━━━━━━━━ 69s 139ms/step - accuracy: 0.8399 - auc: 0.9080 - loss: 0.4174 - val_accuracy: 0.8794 - val_auc: 0.9614 - val_loss: 0.3057 - learning_rate: 2.5000e-04
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8701 - auc: 0.9391 - loss: 0.3456
Epoch 3: val_auc did not improve from 0.94638
200/200 ━━━━━━━━━━━━━━━━━━━━ 41s 139ms/step - accuracy: 0.8701 - auc: 0.9391 - loss: 0.3456 - val_accuracy: 0.8945 - val_auc: 0.9731 - val_loss: 0.2671 - learning_rate: 2.5000e-04
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8743 - auc: 0.9449 - loss: 0.3238
Epoch 4: val_auc did not improve from 0.94638
200/200 ━━━━━━━━━━

In [7]:
import numpy as np
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score, f1_score
import tensorflow as tf
# Evaluate on test set using the best saved model
model.load_weights('best_model.h5')

# 1) Predict probabilities
y_true, y_prob = [], []
for x_batch, y_batch in test_ds:
    y_true.extend(y_batch.numpy())
    y_prob.extend(model.predict(x_batch).flatten())
y_true = np.array(y_true)
y_prob = np.array(y_prob)

# 2) Compute ROC curve and Youden’s J threshold
fpr, tpr, thresholds = roc_curve(y_true, y_prob)
j_idx = np.argmax(tpr - fpr)
best_threshold = thresholds[j_idx]
print(f'Optimal threshold (Youden J): {best_threshold:.3f}')

# 3) Compute binary predictions and metrics
y_pred = (y_prob >= best_threshold).astype(int)
test_acc = accuracy_score(y_true, y_pred)
test_auc = roc_auc_score(y_true, y_prob)
test_f1  = f1_score(y_true, y_pred)

print(f'Test AUC : {test_auc:.4f}')
print(f'Test Acc : {test_acc:.4f}')
print(f'Test F1  : {test_f1:.4f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Optimal threshol